In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from pydataset import data

### Data Cleaning:

tidy data != clean data

- check for outliers
- parse dates
- missing value imputation

**data tidying**: structuring datasets to facilitate analysis

### Data semantics: tidy data

- **Value**: 
    - every value belongs to a variable and an observation.
    - the value that you measure
    
- **Variable**: 
    - a variable contains all values that measure the same underlying attribute (like height, temperature, duration) across units.
    - (ex): measuring the heights of all Florence students- height is variable
    
- **Observation**: an observation contains all values measured on the same unit (like a person, or a day, or a race) across attributes.
    

In [2]:
#pull in data
treatments = pd.read_csv('untidy-data/treatment.csv')
treatments

,Unnamed: 0,treatmenta,treatmentb,treatmentc
0,John Smith,NaN,2,0
1,Jane Doe,16.0,11,3
2,Mary Johnson,3.0,1,4


In [3]:
#rename columns
treatments.columns = ['name', 'a', 'b', 'c']
treatments

,name,a,b,c
0,John Smith,NaN,2,0
1,Jane Doe,16.0,11,3
2,Mary Johnson,3.0,1,4


- **observation**:one person getting a single treatment
- **variables**: treatment (a,b, or c)
- **values**:

In [5]:
# restructure data using 'melt'
treatments = treatments.melt(id_vars=['name'], var_name='treatment', value_name='response')
treatments

<ipython-input-5-42e5b09e733c>:2: FutureWarning: This dataframe has a column name that matches the 'value_name' column name of the resultiing Dataframe. In the future this will raise an error, please set the 'value_name' parameter of DataFrame.melt to a unique name.
  treatments = treatments.melt(id_vars=['name'], var_name='treatment', value_name='response')


,name,treatment,response
0,John Smith,treatment,a
1,Jane Doe,treatment,a
2,Mary Johnson,treatment,a
3,John Smith,treatment,b
4,Jane Doe,treatment,b
5,Mary Johnson,treatment,b
6,John Smith,treatment,c
7,Jane Doe,treatment,c
8,Mary Johnson,treatment,c
9,John Smith,response,NaN


^^ This actually made it HARDER to read

### How do you know you have tidy data:

- Each variable forms a column.
- Each observation forms a row.
- Each cell has a single value.
- data is tabular, i.e. made up of rows and columns
- (ex): employees database, chipotle database, etc

In [8]:
tips = data('tips')
tips.head()
#this is an example of TIDY data

,total_bill,tip,sex,smoker,day,time,size
1,16.99,1.01,Female,No,Sun,Dinner,2
2,10.34,1.66,Male,No,Sun,Dinner,3
3,21.01,3.50,Male,No,Sun,Dinner,3
4,23.68,3.31,Male,No,Sun,Dinner,2
5,24.59,3.61,Female,No,Sun,Dinner,4


### General Ideas
- If the units are the same, maybe they should be in the same column.
- If one column has measurements of different units, it should be spread out
- Should you be able to groupby some of the columns? combine them
- Think: Can I pass this data to seaborn? if yes, you have tidy data
- Can we ask interesting questions and answer them with a groupby? i.e. generally we don't want to be taking row or column averages.

## Fixing MESSY data

#### Reshaping data:
   - Wide data --> Long data format 
       - use (Melt) to fix
       - a lot more columns than rows
            
   - Long data --> Wide Data format 
       - use (pivot_table, unstack) to fix
       - a lot of rows (observations) than columns

Melt Data:
   - start in wide format and turn it into long data
   - long sword and you melt it to be fat
    
Pivot Data:
   - start with long data format and turn it into wide data


_________________________

### Examples:

#### why is it messy: column headers are values, not variable names

In [10]:
df = pd.read_csv('untidy-data/pew.csv')

In [13]:
df.shape

(18, 11)

In [11]:
df.head()

,religion,<$10k,$10-20k,$20-30k,$30-40k,$40-50k,$50-75k,$75-100k,$100-150k,>150k,Don't know/refused
0,Agnostic,27,34,60,81,76,137,122,109,84,96
1,Atheist,12,27,37,52,35,70,73,59,74,76
2,Buddhist,27,21,30,34,33,58,62,39,53,54
3,Catholic,418,617,732,670,638,1116,949,792,633,1489
4,Don’t know/refused,15,14,15,11,10,35,21,17,18,116


#### to fix this, you must create a column called income. for each one, youll have a value

In [19]:
#step 1- melt (will now have many more rows than you started with)
pd.melt(df, id_vars='religion')

,religion,variable,value
0,Agnostic,<$10k,27
1,Atheist,<$10k,12
2,Buddhist,<$10k,27
3,Catholic,<$10k,418
4,Don’t know/refused,<$10k,15
...,...,...,...
175,Orthodox,Don't know/refused,73
176,Other Christian,Don't know/refused,18
177,Other Faiths,Don't know/refused,71
178,Other World Religions,Don't know/refused,8


**went from wide format to long format**

In [17]:
#step 2- rename 
df_tidy= pd.melt(df, id_vars='religion', var_name='income', value_name='count')
df_tidy

,religion,income,count
0,Agnostic,<$10k,27
1,Atheist,<$10k,12
2,Buddhist,<$10k,27
3,Catholic,<$10k,418
4,Don’t know/refused,<$10k,15
...,...,...,...
175,Orthodox,Don't know/refused,73
176,Other Christian,Don't know/refused,18
177,Other Faiths,Don't know/refused,71
178,Other World Religions,Don't know/refused,8


In [21]:
df_tidy.shape

(180, 3)

<div class = "alert alert- block alert-info">
<b>pd.melt arguments<b>
    
- **id_vars** = columns you want to keep (not melt)
- **var_name** = name of new column you created by melting columns
- **value_name** = column name for values</div>


_________________

## Example:

### why is it messy: one variable stored across multiple columns¶

In [24]:
billboard = pd.read_csv('untidy-data/billboard.csv', encoding= 'unicode_escape')
billboard.head()

,year,artist,track,time,genre,date.entered,date.peaked,x1st.week,x2nd.week,x3rd.week,...,x67th.week,x68th.week,x69th.week,x70th.week,x71st.week,x72nd.week,x73rd.week,x74th.week,x75th.week,x76th.week
0,2000,Destiny's Child,Independent Women Part I,3:38,Rock,9/23/00,11/18/00,78,63.0,49.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2000,Santana,"Maria, Maria",4:18,Rock,2/12/00,4/8/00,15,8.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2000,Savage Garden,I Knew I Loved You,4:07,Rock,10/23/99,1/29/00,71,48.0,43.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2000,Madonna,Music,3:45,Rock,8/12/00,9/16/00,41,23.0,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2000,"Aguilera, Christina",Come On Over Baby (All I Want Is You),3:38,Rock,8/5/00,10/14/00,57,47.0,45.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
billboard.shape

(317, 83)

In [31]:
#melt
billboard_melt = pd.melt(billboard, id_vars=[['year', 'artist', 'track', 'time', 'genre', 'date.entered',
       'date.peaked'],
        var_name='week', 
        value_name='rating')

SyntaxError: invalid syntax (<ipython-input-31-c17bf2c5f610>, line 4)

In [32]:
# what is the mean rating (across all weeks) of each track?
billboard_melt.groupby('track').rating.mean()

NameError: name 'billboard_melt' is not defined

_________________________

## Example:

### Why is this data Messy: Multiple variables are stored in one column.

In [47]:
df = pd.DataFrame({
    'name': ['Sally', 'Jane', 'Billy', 'Suzy'],
    'pet': ['dog: max', 'dog: buddy', 'cat: grizabella', 'hamster: fred']
})
df

,name,pet
0,Sally,dog: max
1,Jane,dog: buddy
2,Billy,cat: grizabella
3,Suzy,hamster: fred


In [34]:
#split the string
'dog:max'.split(':')

['dog', 'max']

In [35]:
'dog:max'.split(':')[1]

'max'

In [36]:
#vectorized operation on whole series
df.pet.str.split(':')

0           [dog,  max]
1         [dog,  buddy]
2    [cat,  grizabella]
3      [hamster,  fred]
Name: pet, dtype: object

In [37]:
df.pet.str.split(':')[0][1]

' max'

In [38]:
#create two seperate columns with original dataset
df.pet.str.split(':', expand=True)

,0,1
0,dog,max
1,dog,buddy
2,cat,grizabella
3,hamster,fred


In [48]:
df[['pet', 'pet_name']] =df.pet.str.split(':', expand=True)
df.head()

,name,pet,pet_name
0,Sally,dog,max
1,Jane,dog,buddy
2,Billy,cat,grizabella
3,Suzy,hamster,fred


__________

## Example

### Why is this data Messy? 
Variables are stored in both rows and columns

In [49]:
weather = pd.read_csv('untidy-data/weather.csv')
weather.head()

,id,year,month,element,d1,d2,d3,d4,d5,d6,...,d22,d23,d24,d25,d26,d27,d28,d29,d30,d31
0,MX17004,2010,1,tmax,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.8,NaN
1,MX17004,2010,1,tmin,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.5,NaN
2,MX17004,2010,2,tmax,NaN,27.3,24.1,NaN,NaN,NaN,...,NaN,29.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MX17004,2010,2,tmin,NaN,14.4,14.4,NaN,NaN,NaN,...,NaN,10.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MX17004,2010,3,tmax,NaN,NaN,NaN,NaN,32.1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
weather.shape

(22, 35)

#ideas to make TIDY: 
- we can just have a column called 'day' instead of having 31 columns
- 'tmax' and 'tmin' can be in two columns

In [54]:
#melt the 'days'
weather_long = weather.melt(id_vars=['id', 'year', 'month', 'element'], var_name = 'day', value_name='temp')
weather_long.head()

,id,year,month,element,day,temp
0,MX17004,2010,1,tmax,d1,NaN
1,MX17004,2010,1,tmin,d1,NaN
2,MX17004,2010,2,tmax,d1,NaN
3,MX17004,2010,2,tmin,d1,NaN
4,MX17004,2010,3,tmax,d1,NaN


In [55]:
weather_long.shape

(682, 6)

In [58]:
#now we pivot
weather_long.pivot_table(index=['id','year','month', 'day'], columns='element', values = 'temp').reset_index()

element,id,year,month,day,tmax,tmin
0,MX17004,2010,1,d30,27.8,14.5
1,MX17004,2010,2,d11,29.7,13.4
2,MX17004,2010,2,d2,27.3,14.4
3,MX17004,2010,2,d23,29.9,10.7
4,MX17004,2010,2,d3,24.1,14.4
5,MX17004,2010,3,d10,34.5,16.8
6,MX17004,2010,3,d16,31.1,17.6
7,MX17004,2010,3,d5,32.1,14.2
8,MX17004,2010,4,d27,36.3,16.7
9,MX17004,2010,5,d27,33.2,18.2


<div class = "alert alert- block alert-info">

**pd.pivot_table arguments**
- Index = columns you want to keep (not pivot)
- columns = column you want to pivot
- values = values we want to populate in the new columns
- aggfunct = how you want to aggregate the duplicate rows</div>

_______

## Mini Exercise:

1. read in excel file named dem_score.xls
2. convert the data in tidy format (hint: melt the data)
3. convert the melted dataframe back in wide-format (hint: pivot the data)

In [84]:
#1. read in excel file named dem_score.xls
dem = pd.read_excel('dem_score.xlsx')
dem.head(20)

,country,1952,1957,1962,1967,1972,1977,1982,1987,1992
0,Albania,-9,-9,-9,-9,-9,-9,-9,-9,5
1,Argentina,-9,-1,-1,-9,-9,-9,-8,8,7
2,Armenia,-9,-7,-7,-7,-7,-7,-7,-7,7
3,Australia,10,10,10,10,10,10,10,10,10
4,Austria,10,10,10,10,10,10,10,10,10
5,Azerbaijan,-9,-7,-7,-7,-7,-7,-7,-7,1
6,Belarus,-9,-7,-7,-7,-7,-7,-7,-7,7
7,Belgium,10,10,10,10,10,10,10,10,10
8,Bhutan,-10,-10,-10,-10,-10,-10,-10,-10,-10
9,Bolivia,-4,-3,-3,-4,-7,-7,8,9,9


In [67]:
dem.columns

Index(['country', 1952, 1957, 1962, 1967, 1972, 1977, 1982, 1987, 1992], dtype='object')

In [87]:
#2 convert the data in tidy format (hint: melt the data)
dem_melt = dem.melt(id_vars='country', var_name ='year', value_name='score')
dem_melt.head()

,country,year,score
0,Albania,1952,-9
1,Argentina,1952,-9
2,Armenia,1952,-9
3,Australia,1952,10
4,Austria,1952,10


In [88]:
#new shape after melt
dem_melt.shape

(864, 3)

In [93]:
#3 convert the melted dataframe back in wide-format (hint: pivot the data)
dem_pivot = dem_melt.pivot_table(index='country', columns = 'year', values= 'score')
dem_pivot.head()
#this takes us back to the original data set

year,1952,1957,1962,1967,1972,1977,1982,1987,1992
country,,,,,,,,,
Albania,-9,-9,-9,-9,-9,-9,-9,-9,5
Argentina,-9,-1,-1,-9,-9,-9,-8,8,7
Armenia,-9,-7,-7,-7,-7,-7,-7,-7,7
Australia,10,10,10,10,10,10,10,10,10
Austria,10,10,10,10,10,10,10,10,10


In [91]:
dem_pivot.shape

(96, 9)

In [92]:
dem.shape

(96, 10)

_______

## Mini Exercise 2

In [85]:
sales = pd.read_csv('untidy-data/sales.csv')
sales.head()

,Product,2016 Sales,2016 PPU,2017 Sales,2017 PPU,2018 Sales,2018 PPU
0,A,673,5,231,7,173,9
1,B,259,3,748,5,186,8
2,C,644,3,863,5,632,5
3,D,508,9,356,11,347,14


In [94]:
#first, melt data
sales_melt = sales.melt(id_vars='Product')
sales_melt.head()

,Product,variable,value
0,A,2016 Sales,673
1,B,2016 Sales,259
2,C,2016 Sales,644
3,D,2016 Sales,508
4,A,2016 PPU,5


In [95]:
sales_melt.shape

(24, 3)

In [98]:
#second, split after year- create new columns
sales_melt[['year', 'measure']] = sales_melt.variable.str.split(' ', expand = True)
sales_melt.head()

,Product,variable,value,year,measure
0,A,2016 Sales,673,2016,Sales
1,B,2016 Sales,259,2016,Sales
2,C,2016 Sales,644,2016,Sales
3,D,2016 Sales,508,2016,Sales
4,A,2016 PPU,5,2016,PPU


In [102]:
#third, drop 'variable' column (not specified so it gets dropped automatically)
#pivot on 'measure'
sales_tidy = sales_melt.pivot_table(index=['Product', 'year'], columns = 'measure', values = 'value')
sales_tidy

measure       PPU  Sales
Product year            
A       2016    5    673
        2017    7    231
        2018    9    173
B       2016    3    259
        2017    5    748
        2018    8    186
C       2016    3    644
        2017    5    863
        2018    5    632
D       2016    9    508
        2017   11    356
        2018   14    347

In [103]:
#reset index to make table flat
sales_tidy = sales_melt.pivot_table(index=['Product', 'year'], columns = 'measure', values = 'value').reset_index()
sales_tidy

measure,Product,year,PPU,Sales
0,A,2016,5,673
1,A,2017,7,231
2,A,2018,9,173
3,B,2016,3,259
4,B,2017,5,748
5,B,2018,8,186
6,C,2016,3,644
7,C,2017,5,863
8,C,2018,5,632
9,D,2016,9,508
